<a href="https://colab.research.google.com/github/chinmay-sh/StockAnalysisAI/blob/main/TickerAnalysisAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gradio -q
!pip install yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00


In [44]:
from google.colab import userdata
import yfinance as yf
import gradio as gr
import google.generativeai as genai
from google.colab import userdata
import requests
import math

In [50]:
def predict_stock_price(ticker):
    # Get historical data
    stock_data = yf.Ticker(ticker)
    hist = stock_data.history(period="5d")

    hist_closing_prices = {}
    for key, value in hist["Close"].to_dict().items():
        hist_closing_prices[str(key).split(" ")[0]] = math.floor(value)

    # Get current price
    current_price = stock_data.info['currentPrice']

    # Get news sentiment from Alpha Vantage (replace with actual API call)
    # ...

    # Prepare prompt for LLM
    api_key_name = 'GEMINI_API_KEY'
    system_instructions = """
    You are a stock/cryptocurrency price and news sentiment analysis model.

    Input will be stock/cryptocurrency ticker along with current price, 5 days of historical daily price, news sentiment average from AlphaVantage API for latest 10 news articles, and 10 latest news articles will also be input.

    Your task is to output a summary of those 10 news articles along with estimates of closing price for next 3 days. This estimate should be based on price trends along with news sentiment analysis and knowledge gained from news articles.

    Inputs:
    ===============
    Ticker: [string]
    current_price: [float]
    historical_price_data: [list of historical closing prices for last 5 days week]
    news_sentiment_average_alphavantage: Average of news sentiment scores for latest 10 news articles from AlphaVantage API (x, where x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish)
    news_articles_list: list of Urls of Latest 10 news articles related to ticker from AlphaVantage API.


    Outputs (in JSON format):
    ===============
    future_prices_3_days: [list of predicted closing prices for next 3 days]
    summary_of_news_articles: [summary of latest 10 news articles]

    Example:
    ===============
    Input:
    ticker: "AAPL"
    current_price: 150.0
    historical_price_data: {'2024-10-28': 426,'2024-10-29': 431,'2024-10-30': 432,'2024-10-31': 406,'2024-11-01': 410}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: ["https://example.com/article1", "https://example.com/article2", ...]

    Output:
    future_prices_3_days: {'2024-11-02': 415, '2024-11-03': 419, '2024-11-04': 405}
    summary_of_news_articles: "The latest news articles related to AAPL were ..."
    """
    model = 'gemini-1.5-flash' # @param {type: "string"} ["gemini-1.5-pro", "gemini-1.5-flash"]
    temperature = 0.5 # @param {type: "slider", min: 0, max: 2, step: 0.05}

    api_key = userdata.get(api_key_name)
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel(model, system_instruction=system_instructions)
    config = genai.GenerationConfig(temperature=temperature)

    prompt = f"""
    Input:
    ------
    ticker: {ticker}
    current_price: {stock_data.info['currentPrice']}
    historical_price_data: {hist_closing_prices}
    news_sentiment_average_alphavantage: 0.2
    news_articles_list: ["https://example.com/article1", "https://example.com/article2", ...]
    """

    response = model.generate_content(contents=[prompt], generation_config=config)

    print(response.text)

    # Display results in Gradio UI
    # return f"Current Price: {stock_data.info['regularMarketPrice']}\nAverage Sentiment: {avg_sentiment}\nPredicted Price: {prediction}"

# Create Gradio interface
# iface = gr.Interface(
#     fn=predict_stock_price,
#     inputs="text",
#     outputs="text"
# )

# iface.launch()

In [51]:
predict_stock_price("AAPL")

```json
{
  "future_prices_3_days": {
    "2024-11-02": 220,
    "2024-11-03": 218,
    "2024-11-04": 215
  },
  "summary_of_news_articles": "Based on the latest news articles related to AAPL, the stock is expected to experience a slight decline in the coming days. While the news sentiment is somewhat bullish, with an average sentiment score of 0.2, the recent price trend suggests a downward trajectory. The stock has been steadily declining over the past week, indicating potential selling pressure. However, the decline has been gradual, suggesting a potential for stabilization in the near future. Further analysis of the news articles reveals concerns regarding the company's upcoming earnings report and potential market volatility. It is important to note that these are just predictions based on available data and should not be considered financial advice."
}
```

**Explanation:**

* **Price Trend:** The historical price data shows a consistent downward trend over the past 5 days. This 

In [46]:
aapl = yf.Ticker("AAPL")
aapl.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod